In [25]:
from nsepython import * 

#The Golden Ratio BankNIFTY Strategy
##Scrip = BANKNIFTY

## Golden Number = ((Previous Day High - Previous Day Low) + Opening Range of Today's First 10m))*61.8%

### Buy Above = (Previous Day Close + Golden Number)
### Sell Below = (Previous Day Close - Goldern Number)

#### Stop Loss at 0.5% Target at 2%

#Refactoring
symbol = "BANKNIFTY"
symbol_info = nse_quote_meta(symbol,"latest","Fut")
expiry_date = symbol_info['expiryDate']
expiry_date_formatted = (datetime.datetime.strptime(symbol_info['expiryDate'],"%d-%b-%Y")).date()

##Initializing Broker Connection - AliceBlue
from alice_blue import *

username = ''
password = ''
twoFA=''

client_id = ''
client_secret = ''
redirect_url= ''

access_token = AliceBlue.login_and_get_access_token(username=username, password=password, twoFA=twoFA,  api_secret=client_secret, redirect_url=redirect_url, app_id=client_id)
alice = AliceBlue(username=username, password=password, access_token=access_token, master_contracts_to_download=['NSE', 'BSE', 'MCX', 'NFO'])
symbol_fut = alice.get_instrument_for_fno(symbol = symbol, expiry_date=expiry_date_formatted, is_fut=True, strike=None, is_CE = False)
if(symbol=="BANKNIFTY"):quantity = 25
if(symbol=="NIFTY"):quantity = 75


#Getting All the Necessary Variables
historical_data = nsefetch('https://www.nseindia.com/api/historical/fo/derivatives?&expiryDate='+expiry_date+'&instrumentType=FUTIDX&symbol='+symbol)
historical_data = pd.DataFrame(historical_data['data'])
previous_day_high = historical_data['FH_TRADE_HIGH_PRICE'].iloc[0]
previous_day_low = historical_data['FH_TRADE_LOW_PRICE'].iloc[0]

range_high = symbol_info['highPrice']
range_low = symbol_info['lowPrice']
opening_range = range_high-range_low

golden_number = (float(previous_day_high)-float(previous_day_low)+opening_range)*.618

previous_day_close = symbol_info['prevClose']
buy_above = int(previous_day_close+golden_number)
sell_below = int(previous_day_close-golden_number)

print("Buy "+symbol+" Above: "+str(buy_above)+".")
print("Sell "+symbol+" Below: "+str(sell_below)+".")

#Entering the Trade

while True: 
    
    symbol_ltp = nse_quote_ltp(symbol,"latest","Fut")
    print("Current Value of "+symbol+": " + str(symbol_ltp))
    
    who_triggered = "NONE"
    
    #Amit's Test
    ##symbol_ltp = 21949
    
    if(symbol_ltp>buy_above):
        print("Buy Order executed at: " +str(symbol_ltp)+". Entry Time is "+ str(run_time)+".")
        alice.place_order(TransactionType.Buy,symbol_fut,quantity,OrderType.Market,ProductType.Intraday)
        who_triggered = "BUY"
        stop_loss = symbol_ltp*(.995)
        target = symbol_ltp*(1.02)
    
    if(symbol_ltp<sell_below):
        print("Sell Order executed at: " +str(symbol_ltp)+". Entry Time is "+ str(run_time)+".")
        alice.place_order(TransactionType.Sell,symbol_fut,quantity,OrderType.Market,ProductType.Intraday)
        who_triggered = "SELL"
        stop_loss = symbol_ltp*(1.005)
        target = symbol_ltp*(.98)
    
    if(who_triggered != "NONE"):
        entry_price = run_time
        entry_price = symbol_ltp
        print("Stop Loss is: " + str(stop_loss)+".")
        print("Target is: " + str(target)+".")
        break        
        
    time.sleep(10)

#Managing the Trade

while True:
    symbol_ltp = nse_quote_ltp(symbol,"latest","Fut")
    print("Current Value of "+symbol+": " + str(symbol_ltp))
    
    exit_time = run_time
    exit_price = symbol_ltp
        
    if(who_triggered == "BUY"):
        
        if(symbol_ltp>target):
            print("Target hit at: "+ str(symbol_ltp)+". Exit Time is "+ str(run_time)+".")
            alice.place_order(TransactionType.Sell,symbol_fut,quantity,OrderType.Market,ProductType.Intraday)
            print("Net Profit: "+str(abs(entry_price-exit_price))+" points.")
            break
            
        if(symbol_ltp<stop_loss):
            print("Stop Loss hit at: "+ str(symbol_ltp)+". Exit Time is "+ str(run_time)+".")
            alice.place_order(TransactionType.Sell,symbol_fut,quantity,OrderType.Market,ProductType.Intraday)
            print("Net Loss: "+str(abs(entry_price-exit_price))+" points.")
            break
    
    if(who_triggered == "SELL"):
        
        if(symbol_ltp<target):
            print("Target hit at: "+ str(symbol_ltp)+". Exit Time is "+ str(run_time)+".")
            alice.place_order(TransactionType.Buy,symbol_fut,quantity,OrderType.Market,ProductType.Intraday)
            print("Net Profit: "+str(abs(entry_price-exit_price))+" points.")
            break
            
        if(symbol_ltp>stop_loss):
            print("Stop Loss hit at: "+ str(symbol_ltp)+". Exit Time is "+ str(run_time)+".")
            alice.place_order(TransactionType.Buy,symbol_fut,quantity,OrderType.Market,ProductType.Intraday)
            print("Net Loss: "+str(abs(entry_price-exit_price))+" points.")
            break
    

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.nseindia.com:443
DEBUG:urllib3.connectionpool:https://www.nseindia.com:443 "GET /api/equity-stockIndices?index=SECURITIES%20IN%20F%26O HTTP/1.1" 401 272
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): nseindia.com:80
DEBUG:urllib3.connectionpool:http://nseindia.com:80 "GET / HTTP/1.1" 301 0
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): www.nseindia.com:80
DEBUG:urllib3.connectionpool:http://www.nseindia.com:80 "GET / HTTP/1.1" 301 0
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.nseindia.com:443
DEBUG:urllib3.connectionpool:https://www.nseindia.com:443 "GET / HTTP/1.1" 200 36362
DEBUG:urllib3.connectionpool:https://www.nseindia.com:443 "GET /api/equity-stockIndices?index=SECURITIES%20IN%20F%26O HTTP/1.1" 200 21226
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.nseindia.com:443
DEBUG:urllib3.connectionpool:https://www.nseindia.com:443 "GET /api/qu

Buy BANKNIFTY Above: 22332.
Sell BANKNIFTY Below: 20596.
Current Value of BANKNIFTY: 22292


KeyboardInterrupt: 

In [29]:
alice.place_order(TransactionType.Buy,symbol_fut,quantity,OrderType.Market,ProductType.Intraday)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): ant.aliceblueonline.com:443
DEBUG:urllib3.connectionpool:https://ant.aliceblueonline.com:443 "POST /api/v2/order HTTP/1.1" 200 None


{'status': 'success',
 'message': 'Order placed successfully',
 'data': {'oms_order_id': '201003000000039'}}